In [ ]:
from src.rag.components.generator import LLamaCppGeneratorComponent

In [ ]:
prompt = "You are a french news reporter"
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

In [ ]:
llama_cpp_generator = LLamaCppGeneratorComponent(
    api_url="http://127.0.0.1:8001",
    model_name=model_name,
    prompt=prompt
)

In [ ]:
llama_cpp_generator._ping_api()

In [ ]:
summarization_prompt = """
Give a summary in french of the following document:
{{content}}
 Describes it in a style of a french new paper reporters.

Don't summarize each document separately, the content in all the documents should be summarized.

The summary should be in french not in English
"""

### Download the File

In [ ]:
from src.shared.cloud_storage import BackBlazeCloudStorage

In [ ]:
cloud_storage = BackBlazeCloudStorage(environment="prod")

In [ ]:
from datetime import datetime

In [ ]:
today = datetime.now().strftime("%Y-%m-%d")

In [ ]:
today

In [ ]:
bucket_name = "congonews-clusters"
file_name = f"news-clusters-{today}.csv"

In [ ]:
import pandas as pd

In [ ]:
from tempfile import NamedTemporaryFile

In [ ]:
today_news_file = cloud_storage.download_by_name(
    bucket_name=bucket_name, file_name=file_name)
with NamedTemporaryFile(delete=True, suffix=".csv") as temp_file:
    today_news_file.save_to(temp_file.name)
    today_news_df = pd.read_csv(temp_file.name, index_col=0)

In [ ]:
def select_top_clusters(news_df: pd.DataFrame) -> pd.DataFrame:
    """ select the clusters with the more than two documents """
    cluster_counts = news_df["labels"].value_counts()
    labels_with_more_than_one = cluster_counts[cluster_counts > 1].index
    important_news_df = news_df.loc[news_df.labels.isin(
        labels_with_more_than_one)]
    return important_news_df

In [ ]:
today_news_df = select_top_clusters(today_news_df)

In [ ]:
today_news_df = today_news_df.sort_values(by="labels")

### Prompt

In [ ]:
summaries = []
for id, news_group in today_news_df.groupby("labels"):
    news = news_group.content.str.cat(sep="\n")
    summary = llama_cpp_generator.run(
        template_values={"content": news}, prompt_template=summarization_prompt)
    summaries.append(summary)
    print(summary)
    print("---" * 10)

In [ ]:
today_news_df.labels.nunique()

In [ ]:
today_news_df